In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import random
import os

dataRep = '../data/'
datasetRep = '../data/dataset/'
trainImageRep = '../data/train_images/'
scriptRep = '../'

#Import the scripts
import sys
sys.path.insert(1, os.path.abspath(scriptRep))
import kuzushiji_recognition as kr

unicodeData = pd.read_csv(dataRep+'unicode_translation.csv')
trainData = pd.read_csv(dataRep+'train.csv')
sampleSubData = pd.read_csv(dataRep+'sample_submission.csv')

In [2]:
testFrac = 0.15
valFrac = 0.15

unicodeData = pd.read_csv(dataRep+'unicode_translation.csv')

raw = np.load(datasetRep + 'SegmentationMapsFull.npz')
image = raw['compressed_images'].copy()
maps = raw['maps'].copy()
del raw

index = np.arange(image.shape[0])
np.random.shuffle(index)
image = (image[index].astype(np.float16)/255.0)
maps = (maps[index].astype(np.float16)/255.0)
del index

nTest = int(image.shape[0]*(1.-testFrac))
nVal = int(image.shape[0]*(1.-testFrac-valFrac))
print(image.shape[0], image.shape[1:], nVal, nTest)

3881 (512, 512, 1) 2716 3298


In [3]:
model = kr.createModelUNet((512,512,1), 0.25, 16)

W0918 23:29:34.317200 10956 deprecation.py:506] From D:\Kuzushiji-Recognition-Project\env\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv1a (Conv2D)                 (None, 512, 512, 16) 160         input[0][0]                      
__________________________________________________________________________________________________
drop1a (SpatialDropout2D)       (None, 512, 512, 16) 0           conv1a[0][0]                     
__________________________________________________________________________________________________
conv1b (Conv2D)                 (None, 512, 512, 16) 2320        drop1a[0][0]                     
______________________________________________________________________________________________

__________________________________________________________________________________________________
up_sampling2d_2 (UpSampling2D)  (None, 256, 256, 64) 0           drop7c[0][0]                     
__________________________________________________________________________________________________
conv8a (Conv2DTranspose)        (None, 256, 256, 32) 18464       up_sampling2d_2[0][0]            
__________________________________________________________________________________________________
merge8 (Concatenate)            (None, 256, 256, 64) 0           drop2b[0][0]                     
                                                                 conv8a[0][0]                     
__________________________________________________________________________________________________
conv8b (Conv2D)                 (None, 256, 256, 32) 18464       merge8[0][0]                     
__________________________________________________________________________________________________
drop8b (Sp

In [5]:
kr.trainAndEvaluateModel(kr.createModelUNet((512,512,1), 0.25, 16), '../model/' + 'Unet1.h5', 8, 
                      image, maps, nVal, nTest, lossFunc='sparse_categorical_crossentropy', maxEpochs=75)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv1a (Conv2D)                 (None, 512, 512, 16) 160         input[0][0]                      
__________________________________________________________________________________________________
drop1a (SpatialDropout2D)       (None, 512, 512, 16) 0           conv1a[0][0]                     
__________________________________________________________________________________________________
conv1b (Conv2D)                 (None, 512, 512, 16) 2320        drop1a[0][0]                     
____________________________________________________________________________________________

__________________________________________________________________________________________________
up_sampling2d_7 (UpSampling2D)  (None, 256, 256, 64) 0           drop7c[0][0]                     
__________________________________________________________________________________________________
conv8a (Conv2DTranspose)        (None, 256, 256, 32) 18464       up_sampling2d_7[0][0]            
__________________________________________________________________________________________________
merge8 (Concatenate)            (None, 256, 256, 64) 0           drop2b[0][0]                     
                                                                 conv8a[0][0]                     
__________________________________________________________________________________________________
conv8b (Conv2D)                 (None, 256, 256, 32) 18464       merge8[0][0]                     
__________________________________________________________________________________________________
drop8b (Sp

W0918 23:31:08.026646 10956 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 2716 samples, validate on 582 samples
Epoch 1/75


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Can not squeeze dim[3], expected a dimension of 1, got 2
	 [[{{node metrics_2/acc/Squeeze}}]]
	 [[metrics_2/acc/Identity/_635]]
  (1) Invalid argument: Can not squeeze dim[3], expected a dimension of 1, got 2
	 [[{{node metrics_2/acc/Squeeze}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
import platform
platform.architecture()

In [ ]:
is_64bits = sys.maxsize > 2**32
is_64bits

In [ ]:
import numpy as np
a = (np.random.random((4000,512,512,1))*255).astype(np.uint8).astype(np.float16)

In [ ]:
import sys
print(sys.maxsize / 1e6)